In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:9])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.', 'It was too wet to play.', 'So we sat in the house', 'All that cold cold wet day.', 'I sat there with Sally.', 'We sat there we two.', 'And I said How I wish']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    if len(sentence) > 14 :
        sentence = sentence[0:14]

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

The Cat in the
By Dr. Seuss
The sun did no
It was too wet
So we sat in t
All that cold 
I sat there wi
We sat there w
And I said How
We had somethi


In [3]:
import re    

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) > 14 :
        sentence = sentence[0:14]
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did no <end>',
 '<start> it was too wet <end>',
 '<start> so we sat in t <end>',
 '<start> all that cold <end>',
 '<start> i sat there wi <end>',
 '<start> we sat there w <end>',
 '<start> and i said how <end>',
 '<start> we had somethi <end>']

In [5]:
import tensorflow as tf  

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   7 718 ...   0   0   0]
 [  2 145 420 ...   0   0   0]
 [  2   7 364 ...   0   0   0]
 ...
 [  2 764  34 ...   0   0   0]
 [  2 764  34 ...   0   0   0]
 [  2 764  34 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f0889b882d0>


In [6]:
print(tensor[:3, :10])

[[    2     7   718    18     7     3     0     0     0     0]
 [    2   145   420    50 10975     3     0     0     0     0]
 [    2     7   364   165    35     3     0     0     0     0]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : you
6 : ,
7 : the
8 : and
9 : t
10 : a


In [8]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2   7 718  18   7   3   0   0   0   0   0   0   0   0   0]
[  7 718  18   7   3   0   0   0   0   0   0   0   0   0   0]


sklearn 모듈의 train_test_split()
! 총 데이터의 20%를 평가 데이터셋
src_input(source sentence), tgt_input(target sentence)

In [9]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input , tgt_input, test_size = 0.2, random_state = 20)


In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 15)
Target Train: (140599, 15)


out:

Source Train: (124960, 14)
Target Train: (124960, 14)

In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 15, 12001), dtype=float32, numpy=
array([[[ 3.35209450e-04, -2.34102408e-05,  3.25528563e-05, ...,
          4.81849129e-05, -2.23445648e-04,  7.66201847e-05],
        [ 3.16830177e-04, -1.46754013e-04,  3.04348039e-04, ...,
          1.59945062e-04, -4.19319724e-04,  2.92771409e-04],
        [ 9.85333390e-05, -2.11925435e-04,  5.00932103e-04, ...,
          2.12476982e-04, -3.40106519e-04,  3.92943359e-04],
        ...,
        [-1.10624277e-03,  1.23901805e-03, -2.07391768e-05, ...,
          6.93116162e-04, -6.83723192e-05,  2.35018000e-04],
        [-1.29887776e-03,  1.25069474e-03, -1.86946607e-04, ...,
          6.34386961e-04,  2.74654667e-05,  1.05214778e-04],
        [-1.45313598e-03,  1.22838665e-03, -3.43796273e-04, ...,
          5.68541465e-04,  1.35798764e-04, -2.87712246e-05]],

       [[ 3.35209450e-04, -2.34102408e-05,  3.25528563e-05, ...,
          4.81849129e-05, -2.23445648e-04,  7.66201847e-05],
        [ 7.57521717e-04, -2.19537425e-04,  1

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
#model.fit(dataset, epochs=10)
model.fit(dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 71s 130ms/step - loss: 1.5721
Epoch 2/10
549/549 [==============================] - 69s 125ms/step - loss: 1.2800
Epoch 3/10
549/549 [==============================] - 69s 126ms/step - loss: 1.2016
Epoch 4/10
549/549 [==============================] - 77s 140ms/step - loss: 1.1562
Epoch 5/10
549/549 [==============================] - 73s 133ms/step - loss: 1.1216
Epoch 6/10
549/549 [==============================] - 69s 125ms/step - loss: 1.0930
Epoch 7/10
549/549 [==============================] - 76s 139ms/step - loss: 1.0678
Epoch 8/10
549/549 [==============================] - 69s 125ms/step - loss: 1.0462
Epoch 9/10
549/549 [==============================] - 69s 126ms/step - loss: 1.0266
Epoch 10/10
549/549 [==============================] - 69s 125ms/step - loss: 1.0085


In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [17]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

NameError: name 'lyricist' is not defined